In [1]:
%load_ext autoreload

In [2]:
%load_ext line_profiler

In [3]:
%autoreload 2

In [4]:
import sys

In [5]:
import cupy as cp

In [6]:
import cupyx.scipy.sparse as cpsparse

In [7]:
import pyramid as pr
import numpy as np

In [ ]:
pr

In [ ]:
%pdb on

In [8]:
dim = (55,73,73)
dim_uv = (128, 128)
anglelist = (0, -28, -44, -50, -54, -58, -61, -65)
angles = np.deg2rad(anglelist)
num_angle = len(anglelist)
projectors = []
for angle in angles:
    projectors.append(pr.XTiltProjector(dim, angle, dim_uv))

In [9]:
mag = np.random.randn(*(dim + (3,)))

In [10]:
mag.shape

(55, 73, 73, 3)

In [11]:
x_mag, y_mag, z_mag = mag[..., 2], mag[..., 1], mag[..., 0]

In [12]:
[p.dim_uv for p in projectors]

[(128, 128),
 (128, 128),
 (128, 128),
 (128, 128),
 (128, 128),
 (128, 128),
 (128, 128),
 (128, 128)]

In [13]:
magdata = pr.VectorData(a=2.0,field=cp.asarray([x_mag, y_mag, z_mag]))

In [14]:
projectors[0].dim_uv

(128, 128)

In [15]:
tuple(2 * cp.array((128, 128)) - 1)

(array(255), array(255))

In [16]:
kernel = pr.Kernel(magdata.a, projectors[0].dim_uv, b_0=0.483)

In [26]:
def run_proj_map():
    for k in range(num_angle):
        mag_proj = projectors[k](magdata)
        phasemap = pr.PhaseMapperRDFC(kernel)(mag_proj)
        result = phasemap.phase
    return result

In [31]:
%%timeit
run_proj_map()

23.8 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
result

In [ ]:
%lprun -f pr.XTiltProjector._vector_field_projection run_proj_map()

In [ ]:
%%timeit
run_proj_map()

In [ ]:
# projection for 8 angles: 47 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# total: 86.4 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# pulling the kernel out of the loop: 69.1 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [ ]:
projectors[0].weight

In [ ]:
projectors[0].size_2d

In [ ]:
magdata.field_vec

In [18]:
def _vector_field_projection(weight, vector, size_2d, coeff):
    result = np.zeros(2 * size_2d, dtype=vector.dtype)
    # Go over all possible component projections (z, y, x) to (u, v):
    vec_x, vec_y, vec_z = np.split(vector, 3)
    vec_x_weighted = weight.dot(vec_x)
    vec_y_weighted = weight.dot(vec_y)
    vec_z_weighted = weight.dot(vec_z)
    slice_u = slice(0, size_2d)
    slice_v = slice(size_2d, 2 * size_2d)
    if coeff[0][0] != 0:  # x to u
        result[slice_u] += coeff[0][0] * vec_x_weighted
    if coeff[0][1] != 0:  # y to u
        result[slice_u] += coeff[0][1] * vec_y_weighted
    if coeff[0][2] != 0:  # z to u
        result[slice_u] += coeff[0][2] * vec_z_weighted
    if coeff[1][0] != 0:  # x to v
        result[slice_v] += coeff[1][0] * vec_x_weighted
    if coeff[1][1] != 0:  # y to v
        result[slice_v] += coeff[1][1] * vec_y_weighted
    if coeff[1][2] != 0:  # z to v
        result[slice_v] += coeff[1][2] * vec_z_weighted
    return result

In [19]:
projectors[0].weight.get()

<16384x293095 sparse matrix of type '<class 'numpy.float64'>'
	with 586190 stored elements in Compressed Sparse Row format>

In [20]:
%%timeit
for i in range(8):
    res_ = _vector_field_projection(projectors[0].weight.get(), cp.asnumpy(magdata.field_vec), projectors[0].size_2d, cp.asnumpy(projectors[0].coeff))

85.4 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
def _vector_field_projection_cp(weight, vector, size_2d, coeff):
    result = cp.zeros(2 * size_2d, dtype=vector.dtype)
    # Go over all possible component projections (z, y, x) to (u, v):
    vec_x, vec_y, vec_z = np.split(vector, 3)
    vec_x_weighted = weight.dot(vec_x)
    vec_y_weighted = weight.dot(vec_y)
    vec_z_weighted = weight.dot(vec_z)
    slice_u = slice(0, size_2d)
    slice_v = slice(size_2d, 2 * size_2d)
    if coeff[0][0] != 0:  # x to u
        result[slice_u] += coeff[0][0] * vec_x_weighted
    if coeff[0][1] != 0:  # y to u
        result[slice_u] += coeff[0][1] * vec_y_weighted
    if coeff[0][2] != 0:  # z to u
        result[slice_u] += coeff[0][2] * vec_z_weighted
    if coeff[1][0] != 0:  # x to v
        result[slice_v] += coeff[1][0] * vec_x_weighted
    if coeff[1][1] != 0:  # y to v
        result[slice_v] += coeff[1][1] * vec_y_weighted
    if coeff[1][2] != 0:  # z to v
        result[slice_v] += coeff[1][2] * vec_z_weighted
    return result

In [21]:
def _vector_field_projection_cp(weight, vector, size_2d, coeff):
    result = cp.zeros(2 * size_2d, dtype=vector.dtype)
    # Go over all possible component projections (z, y, x) to (u, v):
    vec_x, vec_y, vec_z = np.split(vector, 3)
    vec_x_weighted = weight.dot(vec_x)
    vec_y_weighted = weight.dot(vec_y)
    vec_z_weighted = weight.dot(vec_z)
    slice_u = slice(0, size_2d)
    slice_v = slice(size_2d, 2 * size_2d)
    if coeff[0][0] != 0:  # x to u
        result[slice_u] += coeff[0][0] * vec_x_weighted
    if coeff[0][1] != 0:  # y to u
        result[slice_u] += coeff[0][1] * vec_y_weighted
    if coeff[0][2] != 0:  # z to u
        result[slice_u] += coeff[0][2] * vec_z_weighted
    if coeff[1][0] != 0:  # x to v
        result[slice_v] += coeff[1][0] * vec_x_weighted
    if coeff[1][1] != 0:  # y to v
        result[slice_v] += coeff[1][1] * vec_y_weighted
    if coeff[1][2] != 0:  # z to v
        result[slice_v] += coeff[1][2] * vec_z_weighted
    return result

In [22]:
import cupyx.scipy.sparse as cpsparse

In [23]:
weight_d = cpsparse.csr_matrix(cpsparse.csr_matrix(projectors[0].weight))

In [24]:
vector_d = cp.array(magdata.field_vec)

In [25]:
%%timeit
for i in range(8):
    res = _vector_field_projection_cp(weight_d, vector_d, projectors[0].size_2d, projectors[0].coeff)
# 3.77 ms ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

6.07 ms ± 530 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f _vector_field_projection_cp _vector_field_projection_cp(weight_d, vector_d, projectors[0].size_2d, projectors[0].coeff)

In [ ]:
weight_d.shape